In [55]:
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import Normalizer
import torch
import mglearn
from collections import Counter
from kiwipiepy import Kiwi, TypoTransformer, TypoDefinition
import re
import pandas as pd
import json
import numpy as np
from kiwipiepy.utils import Stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
normalizer = Normalizer()
clustered_dong = pd.read_json("./data/군집상권데이터.json")
menu_data = pd.read_json("./data/메뉴_추상설명_재료_토큰화.json")
# cafe_menu_data = pd.read_json("./data/카페메뉴.json")

In [59]:
menu_data

menu                                               info  \
0         바닐라 라떼                        따뜻 고소 편안 달콤 향긋 은은 포근 조화 균형감   
1         바질 에이드                            신선 허브 향 상쾌 청량감 깨끗 독특 조화   
2          멜론 소다                  시원하 달콤 과일 향 활기차 어리 시절 달콤쌉싸름 경쾌 기포   
3          크림 라떼                          고급 풍미 따뜻 풍성 밀도 감 감 단맛 마무리   
4          유자 소다             상큼 청량감 과일 향 개운 여름 활력 감귤 향 맑 상쾌하 탄산 새콤하   
...          ...                                                ...   
1313      카라멜 머핀    달콤 고소 촉촉 질감 진하 캐러멜 풍미 클래식 디저트 고급 만족감 따뜻 풍미 깊 사랑   
1314      아몬드 머핀        고소하 달콤 질감 촉촉 아몬드 풍미 건강 디저트 클래식 감성 만족감 화사 심플   
1315  콜드 브루 딜라이트     진하 커피 풍미 크리미함 달콤 밸런스 고급 음료 클래식 감성 만족감 깔끔 농후 사랑   
1316     아몬드 크럼블        고소하 바삭한크럼블토핑 달콤 촉촉 중심 건강 디저트 고급 만족감 화사 풍미 깊   
1317    파인애플 탕후루  상큼 달콤 설탕 코팅 신선 파인애플 풍미 화려하 디저트 열대 감성 고급 만족감 생동...   

                                                 ingred  \
0                          우유, 생크림, 설탕, 소금, 바닐라 추출물, 얼음   
1           오렌지 주스, 라임 주스, 레몬 주스, 설탕, 물, 식용 색소, 젤라틴, 얼음   
2      크림치즈, 설탕, 계란, 사워크림, 바닐라 추출물, 레몬즙, 밀가루, 버터, 소금...   
3      아몬드 가루, 슈가파우더, 흰자, 설탕, 바닐라 추출물, 크림치즈, 쿠키 크럼블,...   
4      아몬드 가루, 슈가파우더, 흰자, 설탕, 바닐라 추출물, 체리 잼, 생크림, 버터...   
...                                                 ...   
1313   박력분, 설탕, 베이킹파우더, 소금, 계란, 우유, 버터, 아몬드 슬라이스, 아몬...   
1314           콜드 브루 커피, 우유, 꿀, 얼음, 바닐라 시럽, 초콜릿 시럽, 생크림   
1315                          고구마, 우유, 꿀, 얼음, 바닐라 아이스크림   
1316                   아몬드, 버터, 밀가루, 설탕, 소금, 계란, 베이킹파우더   
1317                                  파인애플, 설탕, 물, 식용색소   

                                         tokenized_info  
0                           따뜻 고소 편안 달콤 향긋 은은 포근 조화 균형감  
1                               신선 허브 향 상쾌 청량감 깨끗 독특 조화  
2                     시원하 달콤 과일 향 활기차 어리 시절 달콤쌉싸름 경쾌 기포  
3                             고급 풍미 따뜻 풍성 밀도 감 감 단맛 마무리  
4                상큼 청량감 과일 향 개운 여름 활력 감귤 향 맑 상쾌하 탄산 새콤하  
...                                                 ...  
1313    달콤 고소 촉촉 질감 진하 캐러멜 풍미 클래식 디저트 고급 만족감 따뜻 풍미 깊 사랑  
1314        고소하 달콤 질감 촉촉 아몬드 풍미 건강 디저트 클래식 감성 만족감 화사 심플  
1315     진하 커피 풍미 크리미함 달콤 밸런스 고급 음료 클래식 감성 만족감 깔끔 농후 사랑  
1316        고소하 바삭한크럼블토핑 달콤 촉촉 중심 건강 디저트 고급 만족감 화사 풍미 깊  
1317  상큼 달콤 설탕 코팅 신선 파인애플 풍미 화려하 디저트 열대 감성 고급 만족감 생동...  

[1318 rows x 4 columns]

In [42]:
app = Flask(__name__)
CORS(app)  # 모든 도메인에서의 요청 허용

# CORS(app, resources={r"/api/*": {"origins": "http://localhost:8087"}})
# REST API key
KAKAO_API_KEY = "015412fbc48c3a4e31b1926b6adb667e"

# 1. 유사 상권 탐색
# 스프링 부트로부터 사용자가 클릭한 좌표 정보를 전달받아 동을 찾은 후 분석하고 전송
@app.route('/send-coordinates', methods=['POST'])
def receive_coordinates():
    # 클라이언트로부터 받은 데이터(현 위치 위도, 경도)
    data = request.get_json()
    latitude = data.get('latitude')
    longitude = data.get('longitude')
    
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    headers = {"Authorization" : f"KakaoAK {KAKAO_API_KEY}"}
    
    response = requests.get(url, headers=headers)
    
    
    dong_number = get_dong_number(latitude, longitude)
    if(dong_number == None):
        return jsonify({"status" : "error", "message" : "현재 지역의 법정동 번호를 찾을 수 없습니다."})
    
    # 유사 상권의 법정동 번호
    similar_dongs = yusa_dong(clustered_dong, dong_number)
    
    # 스트링 부트로 전달
    spring_response = similar_dong_send_to_spring(similar_dongs)
    # 전송 결과 확인
    if(spring_response.get("status")=="success"):
        return jsonify({"status":"success",
                        "yusadongs":similar_dongs,
                        "message": "flask successed"
                       })
    else:
        return jsonify({
            "status" : "error",
            "yusadongs" : "",
            "message" : spring_response.get("error")
        })



# 스프링 부트로부터 키워드 전달받아 분석
@app.route("/analyze-keyword", methods=["POST"])
def analyze_keyword():
    # 요청 본문 데이터 확인 : 키워드
    data = request.get_json()
    keyword = data.get("keyword", "")
    recommanded = input_simil(menu_data, keyword)

    spring_response = keyword_menu_send_to_spring(keyword, recommanded)
    if (spring_response.get("status") == "success"):
        return jsonify({"status": "success",
                        "keyword": keyword,
                        "recommanded": recommanded
                        })
    else:
        return jsonify({
            "status": "error",
            "keyword": keyword,
            "message": spring_response.get("error")
        })

# 스프링 부트로부터 메뉴 전달받아 분석
@app.route("/analyze-menu", methods=["POST"])
def analyze_menu():
    # 요청 데이터 확인 : 메뉴
    data = request.get_json()
    menu_name = data.get("menu","")
    print(menu_name)
    similar_menus = ingred_sim(menu_data, menu_name)
    
    spring_response = similar_menu_recommanded(similar_menus, menu_name)
    print(spring_response["similarMenus"])
    if(spring_response.get("status")=="success"):
        return jsonify({"status":"success",
                        "menu":menu_name,
                       "similarMenus":similar_menus})
    else:
        return jsonify({"status":"error",
                       "message":spring_response.get("error")})

@app.route("/sanggueon-gongtong")
def sanggueon_gongtong():
    data = request.get_json
    sang1 = data.get("sang1", "")
    sang2 = data.get("sang2", "")
    yusasang_gongtong()


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.219.108:5001
Press CTRL+C to quit
192.168.219.108 - - [02/Jan/2025 16:58:56] "POST /analyze-keyword HTTP/1.1" 200 -


{'status': 'success', 'keyword': '촉촉', 'recommanded': ['유사한 메뉴가 존재하지 않습니다...']}


In [7]:
def data_tokenizing(data):
    data["ingred"] = data["ingred"].replace('\'', '', regex=True)
    kiwi = Kiwi()
    for i in range(len(data["info"])):
        data.iloc[i, 1] = kiwi_tokenizer(data["info"][i], tokenizer=kiwi)
    return data

In [9]:
def ingred_sim(data, menu_name, num=10):
    info_sim = info_simil(data, menu_name, 50)
    if (len(info_sim["menu"]) == 0):
        return ["유사한 메뉴가 존재하지 않습니다..."]

    tfidfv = TfidfVectorizer().fit_transform(info_sim["tokenized_info"])

    ingred_simil = cosine_similarity(tfidfv[0], tfidfv)[0]
    ingred_simil_index = np.flip(np.argpartition(ingred_simil, kth=-1)[:-1])

    if (ingred_simil[ingred_simil_index[0]] < 0.2):
        return ["유사한 메뉴가 존재하지 않습니다..."]
    else:
        return list(info_sim.iloc[ingred_simil_index[:num], 0])

In [11]:
# 메뉴의 설명이 유사한 메뉴 추천
def info_simil(data, menu_name, num=10):
    pick = menu_name
    if(menu_name not in list(data["menu"])):
        print("메뉴 이름이 잘못됨")
    pick_idx = data[data["menu"] == pick].index

    tfidfv = TfidfVectorizer().fit_transform(data["tokenized_info"])

    # 단어의 출현 순서에 영향을 받지 않기 위해 코사인 유사도 사용
    cosine_simil = cosine_similarity(tfidfv[pick_idx], tfidfv)[0]

    # 자기 자신은 제외(가장 값이 큰 요소)하고 값이 높은 순으로 정렬
    simil_index = np.flip(np.argpartition(cosine_simil, kth=-1))

    if (cosine_simil[simil_index[1]] < 0.2):
        return data[data["menu"] == "-1"]
    else:
        return data.iloc[simil_index[:num]]

In [83]:
def input_simil(data, input_data, num=10):
    # 입력 데이터를 토큰화
    input_token = kiwi_tokenizer(input_data)
    
    # 데이터 벡터화
    text_vector = np.array(data["menu"] + " " + data["tokenized_info"])
    
    # input_token을 포함한 전체 벡터 생성
    t_vector = np.insert(text_vector, 0, input_token)
    
    # TF-IDF 벡터화
    tfidfv = TfidfVectorizer().fit_transform(t_vector)
    
    # 코사인 유사도 계산
    cosine_simil = cosine_similarity(tfidfv[0], tfidfv)[0]
    
    # 자기 자신 제외하고 가장 유사한 항목 추출
    simil_index = np.flip(np.argpartition(cosine_simil, kth=-1)[:-1])
    simil_index -= 1  # 인덱스 보정

    print(cosine_simil[simil_index[0] + 1])
    # 유사도가 충분히 높지 않으면 메시지 반환
    if cosine_simil[simil_index[0] + 1] < 0.2:
        return ["유사한 메뉴가 존재하지 않습니다..."]
    else:
        return list(data.iloc[simil_index[:num], 0])

In [99]:
recommanded2 = input_simil(menu_data, "촉촉")
recommanded2

0.1800425110945093


['유사한 메뉴가 존재하지 않습니다...']

In [49]:
from kiwipiepy import Kiwi

# Kiwi 객체 생성
kiwi = Kiwi()

# 텍스트 입력
input_text = "안녕하세요! 파이썬을 배우고 있습니다."

# 토큰화
tokenized = kiwi.tokenize(input_text)

# 결과 출력
for word in tokenized:
    print(word)

Token(form='안녕', tag='NNG', start=0, len=2)
Token(form='하', tag='XSA', start=2, len=1)
Token(form='세요', tag='EF', start=3, len=2)
Token(form='!', tag='SF', start=5, len=1)
Token(form='파이썬', tag='NNP', start=7, len=3)
Token(form='을', tag='JKO', start=10, len=1)
Token(form='배우', tag='VV', start=12, len=2)
Token(form='고', tag='EC', start=14, len=1)
Token(form='있', tag='VX', start=16, len=1)
Token(form='습니다', tag='EF', start=17, len=3)
Token(form='.', tag='SF', start=20, len=1)


In [13]:
def kiwi_tokenizer(sentence, tokenizer=Kiwi(typos="basic")):
    sentence = sentence.replace(" ", "")
    stopwords = Stopwords()
    get_tags = ["XR", "NNG", "VA"]
    sentence = tokenizer.space(sentence)

    kiwi_result = tokenizer.tokenize(
        sentence, normalize_coda=True, stopwords=stopwords)
    token_list = []
    for token in range(len(kiwi_result)):
        if (kiwi_result[token].tag in get_tags):
            token_list.append(kiwi_result[token].form)
    token_str = " ".join(token_list)
    return token_str

In [19]:
def yusa_dong(clustered_dong, dong_pk):
    target_dong_index = clustered_dong[clustered_dong["법정동 번호(PK)"]==dong_pk].index[0]

    same_cluster_dong = clustered_dong[clustered_dong["군집"]==clustered_dong.iloc[target_dong_index,-1]]
    same_cluster_dong.reset_index(drop=True, inplace=True)
    target_dong_index = same_cluster_dong[same_cluster_dong["법정동 번호(PK)"]==dong_pk].index[0]
    
    simil_data = same_cluster_dong.iloc[:,3:-1]
    simil_data_scaled = normalizer.fit_transform(simil_data)

    train_tensor = torch.tensor(simil_data_scaled, dtype=torch.float64)
    euclidean_distances = torch.cdist(train_tensor[target_dong_index].unsqueeze(0), train_tensor).squeeze()
    top_idx = np.argsort(euclidean_distances)[1:10]

    return [int(same_cluster_dong["법정동 번호(PK)"].iloc[int(idx)]) for idx in top_idx]

In [21]:
# 키워드 추천 분석 결과 전달
def keyword_menu_send_to_spring(keyword, recommanded):
    spring_url = "http://localhost:8087/api/receive-keyword-menu"
    payload = {
        "status" : "success",
        "keyword": keyword,
        "recommanded": recommanded
    }
    # json 형식으로 전송
    headers = {"Content-Type": "application/json"}
    print(payload)
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return {"status": "success", "message": "스프링으로 전송 완료"}
    except Exception as e:
        # 예외처리
        print(f"스프링 부트 전송 실패: {str(e)}")
        return {"status": "error", "error": str(e)}

In [23]:
# 분석한 유사 상권 정보 전달
def similar_dong_send_to_spring(similar_dongs):
    spring_url = "http://localhost:8087/api/receive-similar-dongs"
    # 스프링에 dong이라는 이름으로 dong_name 데이터 전송
    payload = {"similarDongs": similar_dongs,
               "status": "success"
              }
    # json 형식으로 전송
    headers = {"Content-Type": "application/json"}
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()  # HTTP 오류 있는지 확인 => 예외 발생
        return {"status": "success", "message": "스프링으로 전송 완료"}
    except Exception as e:
        # 예외처리
        print(f"스프링 전송 실패: {str(e)}")
        return {"status": "error", "error": str(e)}

In [25]:
# 유사 메뉴 분석 결과 스프링으로 전송
def similar_menu_recommanded(similar_menus, menu_name):
    spring_url="http://localhost:8087/api/receive-similar-menus"
    
    payload={"status":"success",
            "similarMenus":similar_menus,
            "menu": menu_name
            }
        
    headers={"Content-Type" : "application/json"}
    print(payload)
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return {"status":"success","message":"스프링으로 전송 완료"}
    except Exception as e:
        print(f"스프링 전송 실패: {str(e)}")
        return {"status" : "error", "error":str(e)}

In [27]:
# 좌표를 통해 법정동 번호 조회
def get_dong_number(latitude, longitude, dong_data=clustered_dong):
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

    response = requests.get(url, headers=headers)

    # 200번 : 통신 성공
    if response.status_code == 200:
        address_info = response.json()
        # 카카오 api에 해당 좌표 정보가 있는지 확인
        if address_info['documents']:
            addr = address_info['documents'][0]['address']['address_name']
            region = addr.split(" ")[0]
            gu = addr.split(" ")[1]
            dong = addr.split(" ")[2]

            region_dong = dong_data[dong_data["지역"].str.contains(region)]
            dong_number = list(
                region_dong[region_dong["동 이름"] == dong].head(1)["법정동 번호(PK)"])[0]
            return dong_number
    return None

In [29]:
# 두 상권 중 다른 상권에만 유독 더 많이 파는 메뉴
def yusasang_only(data, sang1, sang2):
    sang1data = data[data["dong"]==sang1]
    sang2data = data[data["dong"]==sang2]
    
    sang1_count = Counter(sang1data['menu_keyword'])
    sang2_count = Counter(sang2data['menu_keyword'])

    sang1_menu = list(sang1_count.keys())
    sang1_cnt = sang1_count.values()
    sang1_cnt_list = list(sang1_cnt)
    sang1_rat = []
    sang1_sum = 0

    sang2_menu = list(sang2_count.keys())
    sang2_cnt = sang2_count.values()
    sang2_cnt_list = list(sang2_cnt)
    sang2_rat = []
    sang2_sum = 0

    for i in range(len(sang1_cnt)):
        sang1_sum+=sang1_cnt_list[i]
    for i in range(len(sang2_cnt)):
        sang2_sum+=sang2_cnt_list[i]

    sang1_rat = np.array([r/sang1_sum for r in sang1_cnt_list])
    sang2_rat = np.array([r/sang2_sum for r in sang2_cnt_list])

    sang1_most_rat = np.flip(np.argpartition(sang1_rat, kth=-1)[:])
    sang2_most_rat = np.flip(np.argpartition(sang2_rat, kth=-1)[:])

    sang1_most = [sang1_menu[menu_i] for menu_i in sang1_most_rat]
    sang2_most = [sang2_menu[menu_i] for menu_i in sang2_most_rat]

    only_sang2 = []
    popularity = len(sang1_most)//5
    
    for i in range(len(sang2_most)):
        if(sang2_most[i] not in sang1_most[:popularity]):
            only_sang2.append(sang2_most[i])
        if(len(only_sang2) == 5):
            break
    if(len(only_sang2) < 3):
        only_sang2.append("두 상권의 거의 모든 메뉴가 유사합니다.")

    return only_sang2

In [31]:
#두 상권 모두 많이 파는 메뉴
def yusasang_gongtong(data, sang1, sang2):
    sang1data = data[data["dong"]==sang1]
    sang2data = data[data["dong"]==sang2]
    
    sang1_count = Counter(sang1data['menu_keyword'])
    sang2_count = Counter(sang2data['menu_keyword'])

    sang1_menu = list(sang1_count.keys())
    sang1_cnt = sang1_count.values()
    sang1_cnt_list = list(sang1_cnt)
    sang1_rat = []
    sang1_sum = 0

    sang2_menu = list(sang2_count.keys())
    sang2_cnt = sang2_count.values()
    sang2_cnt_list = list(sang2_cnt)
    sang2_rat = []
    sang2_sum = 0

    for i in range(len(sang1_cnt)):
        sang1_sum+=sang1_cnt_list[i]
    for i in range(len(sang2_cnt)):
        sang2_sum+=sang2_cnt_list[i]

    sang1_rat = np.array([r/sang1_sum for r in sang1_cnt_list])
    sang2_rat = np.array([r/sang2_sum for r in sang2_cnt_list])

    sang1_most_rat = np.flip(np.argpartition(sang1_rat, kth=-1)[:])
    sang2_most_rat = np.flip(np.argpartition(sang2_rat, kth=-1)[:])

    sang1_most = [sang1_menu[menu_i] for menu_i in sang1_most_rat]
    sang2_most = [sang2_menu[menu_i] for menu_i in sang2_most_rat]
    
    popularity = len(sang2_most) // 10
    
    gongtong_sang = []
    for i in range(popularity, len(sang1_most)):
        if(sang1_most[i] in sang2_most[:popularity]):
            gongtong_sang.append(sang1_most[i])
        if(len(gongtong_sang)==5):
            break
    if(len(gongtong_sang) < 3):
        gongtong_sang.append("두 상권의 메뉴는 거의 유사하지 않습니다.")
    return gongtong_sang

In [33]:
# 현재 상권에서 주로 파는 메뉴
def sang_menu(data, sang, no_menus=[]):
    sangdata = data[data["dong"]==sang]
    
    sang_count = Counter(sangdata['menu_keyword'])
    sang_most = sang_count.most_common(len(sang_count))
    
    sang_best_menu = []
    for i in range(5, len(sang_most)):
        if(sang_most[i][0] in no_menus):
            continue
        sang_best_menu.append(sang_most[i][0])
        if(len(sang_best_menu)==5):
            break
    if(len(sang_best_menu)<3):
        sang_best_menu.append("해당 상권은 분석할 수 없습니다. 사유 : 상권 내 카페 수 부족")
    return sang_best_menu